In [5]:
# Define the specific crawls we want to use
TARGET_CRAWLS = [
    # These IDs will be updated based on what's available when running the code
    # Format: {"name": "descriptive_name", "id": "CC-MAIN-YYYY-WW"}
    {"name": "February 2025", "id": "CC-MAIN-2025-08"},  # February 2025 (or closest available)
    {"name": "August 2024", "id": "CC-MAIN-2024-33"},    # August 2024 (or closest available)
    {"name": "Fall 2023", "id": "CC-MAIN-2023-40"},      # September/October 2023 (or closest available)
    {"name": "February 2022", "id": "CC-MAIN-2022-05"}   # February 2022 (or closest available)
]

In [6]:
!pip install warc

  Preparing metadata (setup.py) ... done
  Created wheel for warc: filename=warc-0.2.1-py3-none-any.whl size=21792 sha256=a1c7f1ed05ab124c78534017901854ae23b2f557f14a15819ba56a23032135a2
  Stored in directory: /root/.cache/pip/wheels/db/af/cd/9b23cf0a3523d6266147e3fedb83e32d5ce8dd6a150dd96230
Successfully built warc


In [7]:
import requests
import gzip
import io
import re

def stream_warc_paths(url, num_lines=50):
    """
    Stream a gzipped file of WARC paths from Common Crawl, decompress it on the fly,
    and return the first n paths.

    Args:
        url: URL to the gzipped paths file
        num_lines: Number of paths to return (default: 50)

    Returns:
        List of the first num_lines WARC file paths
    """
    # Stream the response without downloading the whole file
    response = requests.get(url, stream=True)

    if response.status_code != 200:
        raise Exception(f"Failed to download file: {response.status_code}")

    # Create a gzip decompressor that can process chunks of data
    decompressor = gzip.GzipFile(fileobj=io.BytesIO(response.content), mode='rb')

    # Read and decode the first num_lines lines
    lines = []
    for i, line in enumerate(decompressor):
        if i >= num_lines:
            break
        lines.append(line.decode('utf-8').strip())

    return lines

def stream_warc_content(warc_path, base_url="https://data.commoncrawl.org/", max_records=5):
    """
    Stream a specific WARC file from Common Crawl, decompress it on the fly,
    and extract the first few web page contents.

    Args:
        warc_path: Path to WARC file (from warc.paths.gz)
        base_url: Base URL for Common Crawl data
        max_records: Maximum number of WARC records to process

    Returns:
        List of dictionaries with WARC record information
    """
    full_url = base_url + warc_path
    print(f"Retrieving WARC file from: {full_url}")

    # Stream the WARC.gz file
    response = requests.get(full_url, stream=True)

    if response.status_code != 200:
        raise Exception(f"Failed to download WARC file: {response.status_code}")

    # Process the compressed data in chunks to avoid memory issues
    # We'll use a small buffer size to demonstrate streaming
    buffer_size = 1024 * 1024  # 1MB chunks

    records = []
    record_count = 0

    # Simple WARC parsing
    in_header = False
    current_headers = {}
    current_content = []
    in_record = False  # Flag to track if we're inside a record

    # Create a stream reader for the gzipped content
    stream = io.BytesIO(response.content)
    decompressor = gzip.GzipFile(fileobj=stream, mode='rb')

    # Buffer for incomplete lines
    buffer = b""

    # Process decompressed data line by line
    while True:
        # Read a chunk of data
        chunk = decompressor.read(buffer_size)
        if not chunk:
            break

        # Combine with any leftover buffer from previous chunk
        data = buffer + chunk

        # Split into lines, keeping the last potentially incomplete line in the buffer
        lines = data.split(b'\r\n')
        buffer = lines.pop() if lines else b""

        # Process each line
        for line in lines:
            line_str = line.decode('utf-8', errors='ignore')

            # WARC record separator - start of a new record
            if line_str.startswith('WARC/1.0'):
                # Save previous record if it exists and we haven't reached max_records
                if current_headers and record_count < max_records and in_record:
                    records.append({
                        'headers': current_headers,
                        'content': ''.join(current_content)
                    })
                    record_count += 1

                # Start a new record
                current_headers = {}
                current_content = []
                in_header = True
                in_record = True
                continue

            # Empty line marks the end of headers
            if in_header and not line_str:
                in_header = False
                continue

            # Process header lines
            if in_header:
                if ': ' in line_str:
                    key, value = line_str.split(': ', 1)
                    current_headers[key] = value
            else:
                # Add to content if we're in a record's body
                if current_headers:
                    current_content.append(line_str + '\n')

            # Check if we've reached max_records
            if record_count >= max_records:
                break

        # Break out of the outer loop if we've reached max_records
        if record_count >= max_records:
            break

    # Add the last record if we haven't reached max_records
    if current_headers and record_count < max_records:
        records.append({
            'headers': current_headers,
            'content': ''.join(current_content)
        })

    return records

def extract_html_content(warc_record):
    """
    Extract HTML content from a WARC response record.

    Args:
        warc_record: Dictionary containing WARC record data

    Returns:
        HTML content or empty string if no HTML content is found
    """
    content = warc_record['content']

    # Check if it's a response record
    if warc_record['headers'].get('WARC-Type') != 'response':
        return ""

    # In a WARC response record, the structure is:
    # 1. WARC headers (already separated)
    # 2. HTTP headers
    # 3. Blank line (\r\n\r\n)
    # 4. HTTP body (HTML content)

    # First, look for the double newline that separates HTTP headers from body
    patterns = ['\r\n\r\n', '\n\n']
    for pattern in patterns:
        # We might need to find the second occurrence if there are multiple header blocks
        parts = content.split(pattern, 1)
        if len(parts) > 1:
            # If we found HTTP headers, return everything after them
            return parts[1]

    # If we can't clearly separate headers, look for HTML start
    html_indicators = ['<!DOCTYPE', '<html', '<HTML', '<?xml']
    for indicator in html_indicators:
        pos = content.find(indicator)
        if pos != -1:
            return content[pos:]

    return content

In [8]:
urls = []
for crawl in TARGET_CRAWLS:
    urls.append(f"https://data.commoncrawl.org/crawl-data/{crawl['id']}/warc.paths.gz")
print(urls)

['https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-08/warc.paths.gz', 'https://data.commoncrawl.org/crawl-data/CC-MAIN-2024-33/warc.paths.gz', 'https://data.commoncrawl.org/crawl-data/CC-MAIN-2023-40/warc.paths.gz', 'https://data.commoncrawl.org/crawl-data/CC-MAIN-2022-05/warc.paths.gz']


In [10]:
num_lines = 4
limit = 1200
crawl_data = {}

try:
    # Get first 5 WARC paths


    for warc_paths_url in urls:
        sites = []
        warc_paths = stream_warc_paths(warc_paths_url, num_lines=num_lines)
        for i, path in enumerate(warc_paths):
            print(f"{i+1}: {path}")

        # Now, get the content from the first WARC file
        if warc_paths:
            for i in range(num_lines):
                # Only process first 2 records to keep the output manageable
                warc_records = stream_warc_content(warc_paths[i], max_records=limit)
                # print(f"\nExtracted {len(warc_records)} records from first WARC file:")
                for i, record in enumerate(warc_records):
                    # print(f"\nRecord {i+1}:")
                    # print(f"WARC Type: {record['headers'].get('WARC-Type', 'unknown')}")
                    # print(f"Target URI: {record['headers'].get('WARC-Target-URI', 'unknown')}")

                    # For response records, try to extract the HTML content
                    if record['headers'].get('WARC-Type') == 'response':
                        html_content = extract_html_content(record)
                        # Print a sample of the content (first 500 chars)
                        content_preview = html_content[:500] + "..." if len(html_content) > 500 else html_content
                        if len(sites) < limit: sites.append(html_content)
                        # print(f"HTML Content Preview:\n{content_preview}")
                    else:
                        # Print a sample of the content (first 500 chars)
                        content_preview = record['content'][:500] + "..." if len(record['content']) > 500 else record['content']
                        # print(f"Content Preview:\n{content_preview}")

                    # print(f"num sites: {len(sites)}")
                    if len(sites) == limit: break
                if len(sites) == limit: break
            crawl_data[warc_paths_url] = sites
            print(f"=== {warc_paths_url} scraping complete. {len(sites)} sites catalogued")
        else:
            print("No WARC paths found.")

except Exception as e:
    print(f"Error: {e}")

1: crawl-data/CC-MAIN-2025-08/segments/1738831951398.56/warc/CC-MAIN-20250206114225-20250206144225-00000.warc.gz
2: crawl-data/CC-MAIN-2025-08/segments/1738831951398.56/warc/CC-MAIN-20250206114225-20250206144225-00001.warc.gz
3: crawl-data/CC-MAIN-2025-08/segments/1738831951398.56/warc/CC-MAIN-20250206114225-20250206144225-00002.warc.gz
4: crawl-data/CC-MAIN-2025-08/segments/1738831951398.56/warc/CC-MAIN-20250206114225-20250206144225-00003.warc.gz
Retrieving WARC file from: https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-08/segments/1738831951398.56/warc/CC-MAIN-20250206114225-20250206144225-00000.warc.gz
Retrieving WARC file from: https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-08/segments/1738831951398.56/warc/CC-MAIN-20250206114225-20250206144225-00001.warc.gz
Retrieving WARC file from: https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-08/segments/1738831951398.56/warc/CC-MAIN-20250206114225-20250206144225-00002.warc.gz
=== https://data.commoncrawl.org/crawl-data/CC-MAI

In [ ]:
from bs4 import BeautifulSoup

def extract_text_from_html(html_content):
    """
    Extract plain text from HTML content, removing all HTML tags.

    Args:
        html_content: String containing HTML content

    Returns:
        Plain text extracted from the HTML
    """
    # Parse HTML with Beautiful Soup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Remove script and style elements - they contain text we don't want
    for script_or_style in soup(["script", "style"]):
        script_or_style.decompose()

    # Get text
    text = soup.get_text()

    # Break into lines and remove leading/trailing whitespace
    lines = (line.strip() for line in text.splitlines())

    # Break multi-headlines into a single line
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))

    # Drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)

    return text

# For use in a Colab notebook:
# 1. Install BeautifulSoup if you haven't already
# !pip install beautifulsoup4

# 2. Use it with your HTML content
for i, url in enumerate(crawl_data):
    i = 1
    print(url)
    for site in crawl_data[url]:
        print(f"\n~~~~ SITE {i} ~~~~")
        html_content = site
        text = extract_text_from_html(html_content)
        i += 1



In [31]:
from bs4 import BeautifulSoup
import json
import os
import re

def extract_text_from_html(html_content):
    """
    Extract plain text from HTML content, removing all HTML tags.

    Args:
        html_content: String containing HTML content

    Returns:
        Plain text extracted from the HTML
    """
    try:
        # Parse HTML with Beautiful Soup
        soup = BeautifulSoup(html_content, 'html.parser')

        # Remove script and style elements - they contain text we don't want
        for script_or_style in soup(["script", "style"]):
            script_or_style.decompose()

        # Get text
        text = soup.get_text()

        # Break into lines and remove leading/trailing whitespace
        lines = (line.strip() for line in text.splitlines())

        # Break multi-headlines into a single line
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))

        # Drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk)

        return text
    except Exception as e:
        print(f"Error extracting text: {e}")
        return ""

def sanitize_filename(url):
    """
    Convert a URL into a safe filename.

    Args:
        url: URL string

    Returns:
        Safe filename string
    """
    print(url)
    len_pre = len("https://data.commoncrawl.org/crawl-data/")
    len_post = len("/warc.paths.gz")
    filename = url[len_pre: len(url) - len_post]

    # Remove protocol and replace special characters
    filename = re.sub(r'^https?://', '', filename)
    # Replace any character that's not alphanumeric, underscore, or dash with underscore
    filename = re.sub(r'[^\w\-]', '_', filename)
    # Truncate if too long
    if len(filename) > 100:
        filename = filename[:100]
    return filename + '.json'

def process_crawl_data(crawl_data, output_dir='crawl_json'):
    """
    Process the crawl data and save each crawl to a separate JSON file.

    Args:
        crawl_data: Dictionary with URLs as keys and lists of HTML content as values
        output_dir: Directory to save the JSON files
    """
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Process each crawl URL
    for url, sites in crawl_data.items():
        # Create a JSON object for this crawl
        crawl_json = {
            "crawl_url": url,
            "sites": []
        }

        print(f"Processing crawl: {url}")

        # Process each site in this crawl
        for i, html_content in enumerate(sites):
            print(f"  Processing site {i+1}/{len(sites)}")

            # Extract text from HTML
            text = extract_text_from_html(html_content)

            # Add site data to the JSON object
            site_data = {
                "site_id": i+1,
                "html_length": len(html_content),
                "text": text
            }

            crawl_json["sites"].append(site_data)

        # Create filename from URL
        filename = sanitize_filename(url)
        filepath = os.path.join(output_dir, filename)

        # Write JSON to file
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(crawl_json, f, ensure_ascii=False, indent=2)

        print(f"Saved crawl data to {filepath}")

In [32]:
# Create a dictionary to organize sites by crawl URL
organized_data = {}
for i, url in enumerate(crawl_data):
    # Initialize the list for this URL if it doesn't exist
    if url not in organized_data:
        organized_data[url] = []

    # Add all sites for this URL
    for site in crawl_data[url]:
        organized_data[url].append(site)

# Process the organized data
process_crawl_data(organized_data)

Processing crawl: https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-08/warc.paths.gz
  Processing site 1/1200
  Processing site 2/1200
  Processing site 3/1200
  Processing site 4/1200
  Processing site 5/1200
  Processing site 6/1200
  Processing site 7/1200
  Processing site 8/1200
  Processing site 9/1200
  Processing site 10/1200
  Processing site 11/1200
  Processing site 12/1200
  Processing site 13/1200
  Processing site 14/1200
  Processing site 15/1200
  Processing site 16/1200
  Processing site 17/1200
  Processing site 18/1200
  Processing site 19/1200
  Processing site 20/1200
  Processing site 21/1200
  Processing site 22/1200
  Processing site 23/1200
  Processing site 24/1200
  Processing site 25/1200
  Processing site 26/1200
  Processing site 27/1200
  Processing site 28/1200
  Processing site 29/1200
  Processing site 30/1200
  Processing site 31/1200
  Processing site 32/1200
  Processing site 33/1200
  Processing site 34/1200
  Processing site 35/1200
  Processin

<ipython-input-31-a6e7f679491e>:18: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  soup = BeautifulSoup(html_content, 'html.parser')


  Processing site 223/1200
  Processing site 224/1200
  Processing site 225/1200
  Processing site 226/1200
  Processing site 227/1200
  Processing site 228/1200
  Processing site 229/1200
  Processing site 230/1200
  Processing site 231/1200
  Processing site 232/1200
  Processing site 233/1200
  Processing site 234/1200
  Processing site 235/1200
  Processing site 236/1200
  Processing site 237/1200
  Processing site 238/1200
  Processing site 239/1200
  Processing site 240/1200
  Processing site 241/1200
  Processing site 242/1200
  Processing site 243/1200
  Processing site 244/1200
  Processing site 245/1200
  Processing site 246/1200
  Processing site 247/1200
  Processing site 248/1200
  Processing site 249/1200
  Processing site 250/1200
  Processing site 251/1200
  Processing site 252/1200
  Processing site 253/1200
  Processing site 254/1200
  Processing site 255/1200
  Processing site 256/1200
  Processing site 257/1200
  Processing site 258/1200
  Processing site 259/1200
 